In [ ]:
# 🔧 STEP 1: Clone Repository và Setup Environment
import os
import sys

# Clone repository từ GitHub với branch specification
REPO_URL = "https://github.com/hoangh-e/dog-emotion-recognition-hybrid.git"
BRANCH_NAME = "main"  # Specify branch explicitly
REPO_NAME = "dog-emotion-recognition-hybrid"

if not os.path.exists(REPO_NAME):
    print(f"📥 Cloning repository from {REPO_URL} (branch: {BRANCH_NAME})")
    !git clone -b {BRANCH_NAME} {REPO_URL}
    print("✅ Repository cloned successfully!")
else:
    print(f"✅ Repository already exists: {REPO_NAME}")

# Change to repository directory
os.chdir(REPO_NAME)
print(f"📁 Current directory: {os.getcwd()}")

# Add to Python path để import modules
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
    print("✅ Added repository to Python path")

# Install required packages
print("📦 Installing dependencies...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install opencv-python-headless pillow pandas tqdm gdown albumentations matplotlib seaborn
!pip install scikit-learn timm ultralytics

# 🎯 Import ResNet từ custom module
try:
    from dog_emotion_classification.resnet import (
        load_resnet_model, 
        predict_emotion_resnet,
        get_resnet_transforms,
        create_resnet_model,
        load_resnet50_model,
        load_resnet101_model,
        predict_emotion_resnet50,
        predict_emotion_resnet101
    )

    # Import utility functions for 3-class conversion
    from dog_emotion_classification.utils import (
        convert_dataframe_4class_to_3class,
        get_3class_emotion_classes,
        EMOTION_CLASSES_3CLASS
    )
    from dog_emotion_classification import EMOTION_CLASSES as PACKAGE_EMOTION_CLASSES

    print("✅ Imported 3-class utility functions")
    print(f"📊 Target emotion classes: {EMOTION_CLASSES_3CLASS}")
    print(f"📦 Package emotion classes: {PACKAGE_EMOTION_CLASSES}")
    print("✅ Successfully imported ResNet module from dog_emotion_classification.resnet")
    print("📋 Available functions:")
    print("   - load_resnet_model()")
    print("   - predict_emotion_resnet()")
    print("   - get_resnet_transforms()")
    print("   - create_resnet_model()")
    print("   - load_resnet50_model(), load_resnet101_model()")
    print("   - predict_emotion_resnet50(), predict_emotion_resnet101()")
except ImportError as e:
    print(f"❌ Failed to import ResNet module: {e}")
    print("Please ensure you're in the repository directory and the module exists.")
    raise


In [ ]:
# Check GPU and setup
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

print(f"🔥 PyTorch version: {torch.__version__}")
print(f"🚀 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎯 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    device = torch.device('cuda')
else:
    print("⚠️ Using CPU - training will be slower")
    device = torch.device('cpu')


In [ ]:
# Download and extract dataset
import zipfile

DATASET_ID = "1ZAgz5u64i3LDbwMFpBXjzsKt6FrhNGdW"
DATASET_ZIP = "cropped_dataset_4k_face.zip"

print("📥 Downloading dog emotion dataset...")
if not os.path.exists(DATASET_ZIP):
    !gdown {DATASET_ID} -O {DATASET_ZIP}
    print(f"✅ Dataset downloaded: {DATASET_ZIP}")
else:
    print(f"✅ Dataset already exists: {DATASET_ZIP}")

# Extract dataset
if not os.path.exists("cropped_dataset_4k_face"):
    print("📂 Extracting dataset...")
    with zipfile.ZipFile(DATASET_ZIP, 'r') as zip_ref:
        zip_ref.extractall(".")
    print("✅ Dataset extracted successfully")

# Dataset paths
data_root = os.path.join("cropped_dataset_4k_face", "Dog Emotion")
labels_csv = os.path.join(data_root, "labels.csv")

print(f"\n📂 Dataset structure:")
emotions = [d for d in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, d))]
print(f"   Emotion classes: {emotions}")

for emotion in emotions:
    emotion_path = os.path.join(data_root, emotion)
    if os.path.isdir(emotion_path):
        count = len([f for f in os.listdir(emotion_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"     {emotion}: {count} images")

print(f"   Labels CSV: {'✅' if os.path.exists(labels_csv) else '❌'} {labels_csv}")


In [ ]:
# Create Dataset class
class DogEmotionDataset(Dataset):
    def __init__(self, root, labels_csv, transform=None):
        self.root = root
        df = pd.read_csv(labels_csv)
        self.items = df[['filename', 'label']].values
        unique_labels = sorted(df['label'].unique())
        self.label2index = {name: i for i, name in enumerate(unique_labels)}
        self.index2label = {i: name for name, i in self.label2index.items()}
        self.transform = transform
        print(f"📊 Dataset: {len(self.items)} samples")
        print(f"🏷️  Classes: {list(self.label2index.keys())}")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        fn, label_str = self.items[idx]
        label_idx = self.label2index[label_str]
        img_path = os.path.join(self.root, label_str, fn)
        
        try:
            img = Image.open(img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img, label_idx
        except Exception as e:
            # Fallback for corrupted images
            img = Image.new('RGB', (224, 224), (0, 0, 0))
            if self.transform:
                img = self.transform(img)
            return img, label_idx

# Create transforms for ResNet (224x224 ImageNet standard)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Check if we should use 3-class or 4-class configuration
print("\n🔍 Checking dataset configuration...")

# Create initial dataset to check original classes
print("\n📊 Loading original dataset...")
original_dataset = DogEmotionDataset(data_root, labels_csv, None)
original_classes = list(original_dataset.label2index.keys())
print(f"   Original classes: {original_classes}")
print(f"   Original samples: {len(original_dataset)}")

# Determine if we need 3-class conversion
if 'sad' in original_classes and len(original_classes) == 4:
    print("\n🔧 Converting to 3-class configuration...")
    
    # Read labels CSV and filter out 'sad' class
    labels_df = pd.read_csv(labels_csv)
    print(f"   Original DataFrame: {len(labels_df)} samples")
    
    # Convert to 3-class by removing 'sad' samples
    filtered_df = convert_dataframe_4class_to_3class(labels_df, 'label')
    
    # Save filtered labels CSV
    filtered_labels_csv = os.path.join(data_root, "labels_3class.csv")
    filtered_df.to_csv(filtered_labels_csv, index=False)
    print(f"   Saved filtered labels to: {filtered_labels_csv}")
    
    # Create 3-class dataset
    dataset = DogEmotionDataset(data_root, filtered_labels_csv, train_transform)
    print("✅ Using 3-class configuration")
else:
    # Use original dataset (could be 3-class or 4-class)
    dataset = DogEmotionDataset(data_root, labels_csv, train_transform)
    print(f"✅ Using original {len(original_classes)}-class configuration")

NUM_CLASSES = len(dataset.label2index)
EMOTION_CLASSES = list(dataset.label2index.keys())

print(f"\n✅ Dataset ready:")
print(f"   Total samples: {len(dataset)}")
print(f"   Number of classes: {NUM_CLASSES}")
print(f"   Emotion classes: {EMOTION_CLASSES}")

# Update configuration
print(f"\n🔧 Training configuration:")
print(f"   Classes: {NUM_CLASSES}")
print(f"   Emotion mapping: {dataset.label2index}")
print(f"   ResNet will be created with num_classes={NUM_CLASSES}")


In [ ]:
# Training function with custom checkpointing
def train_resnet_model(model_name, num_epochs=30, batch_size=16, learning_rate=1e-4):
    """
    Train ResNet model with specific checkpointing strategy:
    - Save best model từ epoch 10
    - Save mỗi 5 epochs từ epoch 10 (10, 15, 20, 25, 30)
    """
    print(f"\n🚀 Training {model_name} for {num_epochs} epochs")
    print("="*60)
    
    # Create model using the custom resnet module with correct num_classes
    if model_name == 'resnet50':
        model = create_resnet_model(architecture='resnet50', num_classes=NUM_CLASSES, pretrained=True)
        print(f"🏗️  Created ResNet50 with ImageNet pretrained weights using custom module")
    elif model_name == 'resnet101':
        model = create_resnet_model(architecture='resnet101', num_classes=NUM_CLASSES, pretrained=True)
        print(f"🏗️  Created ResNet101 with ImageNet pretrained weights using custom module")
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    
    model = model.to(device)
    
    # Print model info
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"📊 Model stats:")
    print(f"   Architecture: {model_name}")
    print(f"   Number of classes: {NUM_CLASSES}")
    print(f"   Emotion classes: {EMOTION_CLASSES}")
    print(f"   Total parameters: {total_params:,}")
    print(f"   Trainable parameters: {trainable_params:,}")
    print(f"   Model size: {total_params * 4 / (1024**2):.1f} MB")
    
    # Create data loader
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, 
                             num_workers=2, pin_memory=True)
    
    # Training setup
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    # Training tracking
    train_losses = []
    train_accuracies = []
    best_acc = 0.0
    best_models = {}  # Store multiple best models
    
    # Create checkpoint directory
    checkpoint_dir = f"checkpoints_{model_name}"
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    print(f"🎯 Training configuration:")
    print(f"   Batch size: {batch_size}")
    print(f"   Learning rate: {learning_rate}")
    print(f"   Optimizer: Adam with weight decay 1e-4")
    print(f"   Scheduler: StepLR(step_size=10, gamma=0.1)")
    print(f"   Checkpoint dir: {checkpoint_dir}")
    print(f"   Device: {device}")
    print(f"   Total batches per epoch: {len(train_loader)}")
    
    start_time = time.time()
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training loop with progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, (images, labels) in enumerate(pbar):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{running_loss/(batch_idx+1):.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
        
        # Calculate epoch metrics
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        # Learning rate step
        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]
        
        # Print epoch summary
        elapsed = time.time() - start_time
        eta = elapsed * (num_epochs - (epoch + 1)) / (epoch + 1) if epoch > 0 else 0
        print(f"Epoch {epoch+1:2d}/{num_epochs} | "
              f"Loss: {epoch_loss:.4f} | "
              f"Acc: {epoch_acc:.4f} ({epoch_acc*100:.2f}%) | "
              f"LR: {current_lr:.2e} | "
              f"Time: {elapsed/60:.1f}m | ETA: {eta/60:.1f}m")
        
        # Checkpointing strategy
        if epoch + 1 >= 10:  # Start saving from epoch 10
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                # Save best model
                best_path = os.path.join(checkpoint_dir, f"best_model.pth")
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'loss': epoch_loss,
                    'accuracy': epoch_acc,
                    'model_name': model_name,
                    'num_classes': NUM_CLASSES,
                    'emotion_classes': EMOTION_CLASSES
                }, best_path)
                print(f"✅ New best model saved: {best_path} (Acc: {epoch_acc:.4f})")
            
            # Save every 5 epochs from epoch 10
            if (epoch + 1) % 5 == 0:
                checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch+1}.pth")
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'loss': epoch_loss,
                    'accuracy': epoch_acc,
                    'model_name': model_name,
                    'num_classes': NUM_CLASSES,
                    'emotion_classes': EMOTION_CLASSES
                }, checkpoint_path)
                best_models[f"epoch_{epoch+1}"] = {
                    'path': checkpoint_path,
                    'accuracy': epoch_acc,
                    'loss': epoch_loss
                }
                print(f"📦 Checkpoint saved: {checkpoint_path}")
    
    total_time = time.time() - start_time
    print(f"\n🎉 Training completed!")
    print(f"   Total time: {total_time/60:.1f} minutes")
    print(f"   Best accuracy: {best_acc:.4f} ({best_acc*100:.2f}%)")
    print(f"   Final accuracy: {train_accuracies[-1]:.4f}")
    print(f"   Saved models: {len(best_models) + 1}")  # +1 for best model
    
    return {
        'model': model,
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'best_acc': best_acc,
        'best_models': best_models,
        'checkpoint_dir': checkpoint_dir,
        'total_time': total_time,
        'num_classes': NUM_CLASSES,
        'emotion_classes': EMOTION_CLASSES
    }


In [ ]:
# Train ResNet50 with pretrained weights
print("🔥 Starting ResNet50 Training")
print("🎯 Strategy: 30 epochs, save best từ epoch 10, save mỗi 5 epochs từ epoch 10")
print(f"🏷️  Training on {NUM_CLASSES} classes: {EMOTION_CLASSES}")

resnet50_results = train_resnet_model(
    model_name='resnet50',
    num_epochs=30,
    batch_size=16,
    learning_rate=1e-4
)

print(f"\n✅ ResNet50 training completed!")
print(f"🎯 Best accuracy: {resnet50_results['best_acc']:.4f} ({resnet50_results['best_acc']*100:.2f}%)")
print(f"⏱️  Total training time: {resnet50_results['total_time']/60:.1f} minutes")
print(f"📁 Checkpoints saved in: {resnet50_results['checkpoint_dir']}")


In [ ]:
# Train ResNet101 with pretrained weights
print("\n🔥 Starting ResNet101 Training")
print("🎯 Strategy: 30 epochs, save best từ epoch 10, save mỗi 5 epochs từ epoch 10")
print(f"🏷️  Training on {NUM_CLASSES} classes: {EMOTION_CLASSES}")

resnet101_results = train_resnet_model(
    model_name='resnet101',
    num_epochs=30,
    batch_size=16,
    learning_rate=1e-4
)

print(f"\n✅ ResNet101 training completed!")
print(f"🎯 Best accuracy: {resnet101_results['best_acc']:.4f} ({resnet101_results['best_acc']*100:.2f}%)")
print(f"⏱️  Total training time: {resnet101_results['total_time']/60:.1f} minutes")
print(f"📁 Checkpoints saved in: {resnet101_results['checkpoint_dir']}")

In [ ]:
# Plot training curves comparison
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(15, 5))

# Plot 1: Training Loss
plt.subplot(1, 3, 1)
epochs = range(1, len(resnet50_results['train_losses']) + 1)
plt.plot(epochs, resnet50_results['train_losses'], 'b-', label='ResNet50', linewidth=2)
plt.plot(epochs, resnet101_results['train_losses'], 'r-', label='ResNet101', linewidth=2)
plt.title('🔥 Training Loss Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Training Accuracy
plt.subplot(1, 3, 2)
plt.plot(epochs, [acc*100 for acc in resnet50_results['train_accuracies']], 'b-', label='ResNet50', linewidth=2)
plt.plot(epochs, [acc*100 for acc in resnet101_results['train_accuracies']], 'r-', label='ResNet101', linewidth=2)
plt.title('🎯 Training Accuracy Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Best Accuracy Comparison
plt.subplot(1, 3, 3)
models = ['ResNet50', 'ResNet101']
accuracies = [resnet50_results['best_acc']*100, resnet101_results['best_acc']*100]
colors = ['#3498db', '#e74c3c']
bars = plt.bar(models, accuracies, color=colors, alpha=0.8)
plt.title('🏆 Best Accuracy Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Accuracy (%)')
plt.ylim(0, 100)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{acc:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Print detailed comparison
print("\n" + "="*80)
print("🏆 FINAL TRAINING RESULTS COMPARISON")
print("="*80)
print(f"📊 Dataset: {NUM_CLASSES} classes ({EMOTION_CLASSES})")
print(f"🖼️  Total samples: {len(dataset)}")
print(f"⚙️  Training: 30 epochs, batch_size=16, lr=1e-4")
print()
print("🔵 ResNet50 Results:")
print(f"   Best Accuracy: {resnet50_results['best_acc']:.4f} ({resnet50_results['best_acc']*100:.2f}%)")
print(f"   Final Accuracy: {resnet50_results['train_accuracies'][-1]:.4f} ({resnet50_results['train_accuracies'][-1]*100:.2f}%)")
print(f"   Training Time: {resnet50_results['total_time']/60:.1f} minutes")
print(f"   Checkpoints: {len(resnet50_results['best_models']) + 1} models saved")
print()
print("🔴 ResNet101 Results:")
print(f"   Best Accuracy: {resnet101_results['best_acc']:.4f} ({resnet101_results['best_acc']*100:.2f}%)")
print(f"   Final Accuracy: {resnet101_results['train_accuracies'][-1]:.4f} ({resnet101_results['train_accuracies'][-1]*100:.2f}%)")
print(f"   Training Time: {resnet101_results['total_time']/60:.1f} minutes")
print(f"   Checkpoints: {len(resnet101_results['best_models']) + 1} models saved")
print()
print("📈 Performance Analysis:")
acc_diff = resnet101_results['best_acc'] - resnet50_results['best_acc']
time_diff = resnet101_results['total_time'] - resnet50_results['total_time']
print(f"   Accuracy Difference: {acc_diff:+.4f} ({acc_diff*100:+.2f}%)")
print(f"   Training Time Difference: {time_diff/60:+.1f} minutes")
if acc_diff > 0:
    print(f"   🏆 ResNet101 performed better by {acc_diff*100:.2f}%")
elif acc_diff < 0:
    print(f"   🏆 ResNet50 performed better by {abs(acc_diff)*100:.2f}%")
else:
    print(f"   🤝 Both models achieved the same accuracy")
print("="*80)

In [ ]:
# Evaluation functions
def evaluate_model(model, test_loader, emotion_classes):
    """Evaluate model on test set"""
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = correct / total
    return accuracy, all_predictions, all_labels

def load_best_model(checkpoint_path, model_name, num_classes):
    """Load the best saved model"""
    if model_name == 'resnet50':
        model = create_resnet_model(architecture='resnet50', num_classes=num_classes, pretrained=False)
    elif model_name == 'resnet101':
        model = create_resnet_model(architecture='resnet101', num_classes=num_classes, pretrained=False)
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    
    print(f"✅ Loaded {model_name} from {checkpoint_path}")
    print(f"   Epoch: {checkpoint['epoch']}")
    print(f"   Training Accuracy: {checkpoint['accuracy']:.4f} ({checkpoint['accuracy']*100:.2f}%)")
    print(f"   Training Loss: {checkpoint['loss']:.4f}")
    
    return model

# Create test dataset for evaluation
test_dataset = DogEmotionDataset(data_root, 
                                filtered_labels_csv if 'filtered_labels_csv' in locals() else labels_csv, 
                                val_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"📊 Test dataset ready: {len(test_dataset)} samples")
print(f"🔬 Test loader: {len(test_loader)} batches")

In [ ]:
# Load and evaluate best ResNet50 model
print("🔵 Evaluating Best ResNet50 Model")
print("="*50)
best_resnet50_path = os.path.join(resnet50_results['checkpoint_dir'], "best_model.pth")
best_resnet50 = load_best_model(best_resnet50_path, 'resnet50', NUM_CLASSES)
resnet50_test_acc, resnet50_preds, resnet50_labels = evaluate_model(best_resnet50, test_loader, EMOTION_CLASSES)

print(f"\n🎯 ResNet50 Test Results:")
print(f"   Test Accuracy: {resnet50_test_acc:.4f} ({resnet50_test_acc*100:.2f}%)")
print(f"   Training Best: {resnet50_results['best_acc']:.4f} ({resnet50_results['best_acc']*100:.2f}%)")
print(f"   Difference: {(resnet50_test_acc - resnet50_results['best_acc'])*100:+.2f}%")

print("\n" + "="*50)

# Load and evaluate best ResNet101 model
print("🔴 Evaluating Best ResNet101 Model")
print("="*50)
best_resnet101_path = os.path.join(resnet101_results['checkpoint_dir'], "best_model.pth")
best_resnet101 = load_best_model(best_resnet101_path, 'resnet101', NUM_CLASSES)
resnet101_test_acc, resnet101_preds, resnet101_labels = evaluate_model(best_resnet101, test_loader, EMOTION_CLASSES)

print(f"\n🎯 ResNet101 Test Results:")
print(f"   Test Accuracy: {resnet101_test_acc:.4f} ({resnet101_test_acc*100:.2f}%)")
print(f"   Training Best: {resnet101_results['best_acc']:.4f} ({resnet101_results['best_acc']*100:.2f}%)")
print(f"   Difference: {(resnet101_test_acc - resnet101_results['best_acc'])*100:+.2f}%")

In [ ]:
# Detailed classification report
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Classification reports
print("\n" + "="*80)
print("📊 DETAILED CLASSIFICATION REPORTS")
print("="*80)

print("\n🔵 ResNet50 Classification Report:")
print("-" * 60)
print(classification_report(resnet50_labels, resnet50_preds, target_names=EMOTION_CLASSES, digits=4))

print("\n🔴 ResNet101 Classification Report:")
print("-" * 60)
print(classification_report(resnet101_labels, resnet101_preds, target_names=EMOTION_CLASSES, digits=4))

# Confusion matrices
plt.figure(figsize=(15, 6))

# ResNet50 confusion matrix
plt.subplot(1, 2, 1)
cm_resnet50 = confusion_matrix(resnet50_labels, resnet50_preds)
sns.heatmap(cm_resnet50, annot=True, fmt='d', cmap='Blues', 
           xticklabels=EMOTION_CLASSES, yticklabels=EMOTION_CLASSES)
plt.title('🔵 ResNet50 Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

# ResNet101 confusion matrix
plt.subplot(1, 2, 2)
cm_resnet101 = confusion_matrix(resnet101_labels, resnet101_preds)
sns.heatmap(cm_resnet101, annot=True, fmt='d', cmap='Reds',
           xticklabels=EMOTION_CLASSES, yticklabels=EMOTION_CLASSES)
plt.title('🔴 ResNet101 Confusion Matrix', fontsize=14, fontweight='bold')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()

# Final summary
print("\n" + "="*80)
print("🏆 FINAL MODEL COMPARISON SUMMARY")
print("="*80)
print(f"📊 Dataset: {NUM_CLASSES}-class Dog Emotion Classification")
print(f"🏷️  Classes: {EMOTION_CLASSES}")
print(f"🖼️  Total samples: {len(dataset)} (training) + {len(test_dataset)} (test)")
print(f"⚙️  Configuration: 30 epochs, ImageNet pretrained, Adam optimizer")
print()
print("📈 Training Performance:")
print(f"   ResNet50  - Best: {resnet50_results['best_acc']*100:.2f}% | Time: {resnet50_results['total_time']/60:.1f}m")
print(f"   ResNet101 - Best: {resnet101_results['best_acc']*100:.2f}% | Time: {resnet101_results['total_time']/60:.1f}m")
print()
print("🎯 Test Performance:")
print(f"   ResNet50  - Test: {resnet50_test_acc*100:.2f}%")
print(f"   ResNet101 - Test: {resnet101_test_acc*100:.2f}%")
print()
print("🏆 Winner:")
if resnet101_test_acc > resnet50_test_acc:
    winner = "ResNet101"
    diff = (resnet101_test_acc - resnet50_test_acc) * 100
    print(f"   🥇 ResNet101 wins by {diff:.2f}% on test set")
elif resnet50_test_acc > resnet101_test_acc:
    winner = "ResNet50"
    diff = (resnet50_test_acc - resnet101_test_acc) * 100
    print(f"   🥇 ResNet50 wins by {diff:.2f}% on test set")
else:
    winner = "Tie"
    print(f"   🤝 Both models achieved the same test accuracy")

print()
print("💾 Saved Models:")
print(f"   ResNet50:  {resnet50_results['checkpoint_dir']}/best_model.pth")
print(f"   ResNet101: {resnet101_results['checkpoint_dir']}/best_model.pth")
print("\n✅ Training and evaluation completed successfully!")
print("="*80)

In [ ]:
# Example usage of trained models
print("\n" + "="*60)
print("🧪 MODEL USAGE EXAMPLES")
print("="*60)

# Function to predict single image
def predict_single_image(model, image_path, emotion_classes, transform):
    """Predict emotion for a single image"""
    try:
        # Load and preprocess image
        image = Image.open(image_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # Predict
        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)
            confidence, predicted = torch.max(probabilities, 1)
            
        predicted_emotion = emotion_classes[predicted.item()]
        confidence_score = confidence.item()
        
        return predicted_emotion, confidence_score, probabilities.cpu().numpy()[0]
    except Exception as e:
        print(f"Error predicting image {image_path}: {e}")
        return None, 0, None

# Show how to use the custom module functions
print("\n📝 Using Custom Module Functions:")
print("\n# Load ResNet50 model from checkpoint")
print(f"resnet50_model = load_resnet50_model('{best_resnet50_path}')")
print("\n# Load ResNet101 model from checkpoint")
print(f"resnet101_model = load_resnet101_model('{best_resnet101_path}')")
print("\n# Predict with ResNet50")
print("emotion, confidence = predict_emotion_resnet50(image_path, resnet50_model)")
print("\n# Predict with ResNet101")
print("emotion, confidence = predict_emotion_resnet101(image_path, resnet101_model)")

print("\n🎯 Model Performance Summary:")
print(f"   Best ResNet50:  {resnet50_test_acc*100:.2f}% test accuracy")
print(f"   Best ResNet101: {resnet101_test_acc*100:.2f}% test accuracy")
print(f"   Classes: {EMOTION_CLASSES}")
print(f"   Input size: 224x224 RGB")
print(f"   Preprocessing: ImageNet normalization")

print("\n✅ Models are ready for inference!")
print("   Use the checkpoint files for deployment or further fine-tuning.")
print("   Both models use ImageNet pretrained weights and are optimized for dog emotion recognition.")
print("="*60)